In [9]:
# =============================================================================
#ユーザーが任意のファイルを指定し、読み取り操作のボタンを押すと、
#合計金額がメッセージボックスに返される。
# =============================================================================

import os
import sys
import tkinter
import tkinter.filedialog
from tkinter import messagebox
import camelot
import functools
import pandas as pd
import traceback

# =============================================================================
#グローバル変数の定義
# =============================================================================

# 現在のユーザー名を取得
user_name = os.getlogin()

# 選択されていない場合の判定に利用
file_path = ""

# =============================================================================
#関数 file_select で取得した変数file_pathを引数として使用する
# =============================================================================

#ファイルを選択する関数
def file_select():
    global file_path
    idir= os.path.join(user_name, "Desktop")
    filetype = [("PDF","*.pdf"), ("all", "*")]
    file_path = tkinter.filedialog.askopenfilename(filetypes = filetype, initialdir = idir)
    input_box.insert(tkinter.END, file_path)
    return file_path

#入金票の合計額を算出する関数
def pdf_ttl(event, input_file):
    if not input_file == "":
        table_area = ["70, 560, 120, 520"]
        tables = camelot.read_pdf(input_file, flavor="stream",table_areas=table_area, pages="1-end")
        
        dfs = [table.df for table in tables]
        final_df = pd.concat(dfs, ignore_index=True)
        #データ型が"object"のため、"int64"へと変更する
        final_df = final_df.astype("int64")
        ttl = final_df.sum()
                     
        messagebox.showinfo("成功", "合計金額は{:,.0f}円です".format(ttl[0]))
    else:
        # エラーが発生した場合、以下のメッセージボックスを表示します。
        messagebox.showinfo("失敗","ファイルを指定してください")

# =============================================================================
# GUI
# =============================================================================

#GUIの基本サイズ
root = tkinter.Tk()
root.title(u"入金票合計額算出")
root.geometry("500x200")

#選択したファイルを削除する関数
def DeleteEntryValue(event):
    #エントリーの中身を削除
    input_box.delete(0, tkinter.END)

#ファイルを選択するbox
input_box = tkinter.Entry(width=80)
input_box.place(x=10, y=10)

#ファイルを選択する参照ボタン
button_ref = tkinter.Button(text="ファイル選択", command=file_select)
button_ref.place(x=10, y=35)

#削除ボタン
button_del = tkinter.Button(text=u'削除', width= 8)
button_del.bind("<Button-1>", DeleteEntryValue) 
#左クリック（<Button-1>）されると，DeleteEntryValue関数を呼び出すようにバインド
button_del.place(x=85, y=35)

#結合使用時の注意ラベル
Static1 = tkinter.Label(text = u"合計額を表示したい入金票を選択してください", foreground='#ff0000')
Static1.place(x=170, y=38)

#ttl ボタン
button = tkinter.Button(text=u'合計金額算出', command=functools.partial(pdf_ttl, None, file_path), width= 55, height=5)
button.bind("<Button-1>", lambda event: pdf_ttl(event, file_path)) 
button.place(x=50, y=80)

root.mainloop()